In [1]:
!pip install beautifulsoup4 
!pip install google
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.0 MB/s eta 0:00:00


In [2]:
import data
import csv
import pandas as pd


In [3]:
# read in txt data as list
def readInFile(filename = 'googlesearch-aups.txt'):
  with open(filename) as file:
      aup_data = [line.rstrip() for line in file]

  return aup_data[1:] # get rid of the header

# Url Collection

In [6]:
# for each url we find their snapshots for the past 10 years, one for each year
def collectURL(aup_data):
  '''
  url_snapshots_dict: {
      url: (year, snapshot)
  }
  '''
  url_snapshots_dict = {}

  for url in aup_data:
    try:
      snapshots, timestamps = data.get_snapshots(url)
      snapshot_within_10_years = []
      listed_years = [] # a list that keeps the recorded years since we only want 1 snapshot each year
      for i in range(len(timestamps)):
        year = int(timestamps[i][:4])
        if year in listed_years:
          continue
        # if the timestamp is within 10 years, append corresponding snapshot
        if year >= 2013 and year <= 2023:
          snapshot_within_10_years.append((year, snapshots[i]))
          listed_years.append(year)
      url_snapshots_dict[url] = snapshot_within_10_years
    except Exception as inst: 
      print(f"Exception when dealing with {url}")
      print(inst)
      continue
  
  return url_snapshots_dict

In [7]:
def writeToCSV(url_snapshots_dict):
  try:
    df = pd.DataFrame(url_snapshots_dict.items(), columns=['aup-url', 'snapshots'])
    df.to_csv('snapshots_from_googlesearch.csv', index=False, header=True)
    return True
  except:
    print("Error wrting to CSV")
    return False

In [9]:
aup_data = readInFile()
url_snapshots_dict = collectURL(aup_data)
writeToCSV(url_snapshots_dict)

Exception when dealing with https://www.egress.com/legal/acceptable-use-policy
[Errno Expecting value] org.archive.util.io.RuntimeIOException: org.archive.wayback.exception.AdministrativeAccessControlException: Blocked Site Error
: 0
Exception when dealing with https://www.contractscounsel.com/t/us/acceptable-use-policy
HTTPConnectionPool(host='web.archive.org', port=80): Max retries exceeded with url: /cdx/search/cdx?url=https://www.contractscounsel.com/t/us/acceptable-use-policy&fl=timestamp,digest,length,original&output=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f631d4f59c0>: Failed to establish a new connection: [Errno 111] Connection refused'))


True

# Scrap AUP content

In [10]:
# Now, we start to get AUP content using the urls
def scrapAUP(url_snapshots_dict):
  idx = 0

  for url, tuples in url_snapshots_dict.items():
    if len(tuples) == 0:
      continue
    years = [t[0] for t in tuples]
    snapshots = [t[1] for t in tuples]
    for i in range(len(snapshots)):
      fname = f"aup/{idx}-{years[i]}.txt"
      data.get_aup_content(snapshots[i], fname)
    
    idx += 1


In [ ]:
scrapAUP(url_snapshots_dict)

In [ ]:
!tar -zcvf aup.tar.gz aup